In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
import tensorflow_hub as hub

In [3]:
from timeit import default_timer as timer
import numpy as np
from scipy.spatial.distance import cosine

In [4]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" 

In [5]:
start = timer()
# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url)
end = timer()
print("Time taken: {}".format(end-start))

Time taken: 1.4653810629999997


In [6]:
# Compute a representation for each message, showing various lengths supported.
word = "knowledge graph"
word2 = "ontology"
word3 = "searchengine"
word4 = "donald trump"
word5 = "test staging today"
sentence = "Entities and relations need to be defined well to build good graph representations."
sentence2 = "General knowledge is a subject that I learnt in school and I think it is cool."
paragraph = (
    "Universal Sentence Encoder embeddings also support short paragraphs. "
    "There is no hard limit on how long the paragraph is. Roughly, the longer "
    "the more 'diluted' the embedding will be.")
messages = [word, sentence, sentence2, paragraph, word2, word3, word4, word5]

# Reduce logging output.
tf.logging.set_verbosity(tf.logging.ERROR)

with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    start = timer()
    message_embeddings = session.run(embed(messages))
    end = timer()
    print("Time taken: {}".format(end-start))
    for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
        print("Message: {}".format(messages[i]))
        print("Embedding size: {}".format(len(message_embedding)))
        message_embedding_snippet = ", ".join(
            (str(x) for x in message_embedding[:3]))
        print("Embedding: [{}, ...]".format(message_embedding_snippet))
        dist = cosine(np.array(message_embeddings[0]), message_embedding)
        print("Cosine similarity: {}\n".format(1-dist))

Time taken: 2.3152105150000004
Message: knowledge graph
Embedding size: 512
Embedding: [0.007142363116145134, 0.012228635139763355, -0.003253899747505784, ...]
Cosine similarity: 1.0000000208608009

Message: Entities and relations need to be defined well to build good graph representations.
Embedding size: 512
Embedding: [0.04317519813776016, -0.03976033255457878, -0.03193526342511177, ...]
Cosine similarity: 0.43334049617738213

Message: General knowledge is a subject that I learnt in school and I think it is cool.
Embedding size: 512
Embedding: [-0.03692379966378212, 0.037080686539411545, 0.0314854234457016, ...]
Cosine similarity: 0.45122039014654813

Message: Universal Sentence Encoder embeddings also support short paragraphs. There is no hard limit on how long the paragraph is. Roughly, the longer the more 'diluted' the embedding will be.
Embedding size: 512
Embedding: [0.018790947273373604, 0.045365139842033386, -0.02001088857650757, ...]
Cosine similarity: 0.21234554389034144

M

## Test on word graph

In [6]:
def sort_by_value(item_list, order='desc'):
    """
    A utility function to sort lists by their value.
    Args:
        item_list:
        order:

    Returns:

    """

    if order == 'desc':
        sorted_list = sorted(item_list, key=lambda x: (x[1], x[0]), reverse=True)
    else:
        sorted_list = sorted(item_list, key=lambda x: (x[1], x[0]), reverse=False)

    return sorted_list

In [7]:
from graphrank import GraphRank, GraphUtils, TextPreprocess
import networkx as nx
import community

W0801 10:29:34.621936 4661413312 util.py:15] [E050] Can't find model 'vendor/en_core_web_sm/en_core_web_sm-2.1.0'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/shashank/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
gr = GraphRank()
tp = TextPreprocess()
utils = GraphUtils()

In [9]:
meeting_data = "data/01DB8DEW0YFYK0ZBP2Q3XR2YT1_5f89df0e-3631-4c64-a7ff-3bf0264c830f"

In [10]:
meeting_graph = nx.read_gpickle(meeting_data)

In [11]:
segment_list = []
for node, attr in meeting_graph.nodes(data=True):
    if attr.get("label") == "segmentId":
        segment_list.append(attr.get("text"))

In [20]:
word_graph.clear()

In [36]:
# word_graph.clear()
for i, text in enumerate(segment_list):
    original_tokens, pos_tuple, filtered_pos_tuple = tp.preprocess_text(text, filter_by_pos=True, stop_words=False)
    word_graph = gr.build_word_graph(input_pos_text=pos_tuple, window=2)
    sub_keyphrases = gr.get_keyphrases(graph_obj=word_graph, input_pos_text=pos_tuple)
    print(text)
    print()
    print(sub_keyphrases)
    print()
    if i == 1:
        break


I was born in involved like many other engineers. I grew reading section how traffic was actually very involved with this house and crackers and little that I know that this works a major invoice on my life did you time section to do you remember the moment when the while basically saving the world asks careful for something computer make itself and the machine just does it and is sales how why the real want know like this example for the okay Google and ask the application service mostly requirement and get the job done just like that today I want to talk about technology based on other which one is a small step in that direction when happens we have an opportunity to like your users and bring more engagement to Europe. However, we can't do it need help as Google. We have been working on important organizing the also information to make a university. The access to help with that we build the knowledge that knowledge information about ent*ties and their relationships one of the interes

In [37]:
word_graph.number_of_edges()

95

In [52]:
def get_embedding(session, word):
    message_embeddings = session.run(embed(word))
    dist = cosine(np.array(message_embeddings[0]), np.array(message_embeddings[1]))

    return dist

In [53]:
def get_edge_weight (session, word1 , word2):
    word_list = [word1, word2]
    cosine_dist = get_embedding(session, word_list)
    try:
        return 1-cosine_dist
    except KeyError:
        print("word not found: {}--{}".format(word1, word2))
        return 0

In [54]:
in_vocab_count = 0
out_vocab_count = 0

with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])

    for i, (node1, node2, attr) in enumerate(word_graph.edges.data()):
        start = timer()
        if attr.get("edge_emb_wt") is None:
            emb_edge_weight = get_edge_weight(session, node1.lower(), node2.lower())
            if emb_edge_weight == 0:
                out_vocab_count += 1
            else:
                in_vocab_count += 1
            word_graph.add_edge(node1, node2, edge_emb_wt=emb_edge_weight)
        else:
            continue

        end = timer()
        print("Words seen: {}/{}; Time taken: {}".format(i, len(word_graph.edges.data()), end-start))

print("###########################")
print("Total in_vocab_words = {}".format(in_vocab_count))
print("Total out_of_vocab_words = {}".format(out_vocab_count))
print("Percentage of out_of_vocab = {}".format((out_vocab_count/in_vocab_count)*100))

Words seen: 3/95; Time taken: 89.08197683700018
Words seen: 4/95; Time taken: 75.70530546700138
Words seen: 5/95; Time taken: 70.00495481100006
Words seen: 6/95; Time taken: 72.36020541899961
Words seen: 7/95; Time taken: 92.88816919900091
Words seen: 8/95; Time taken: 116.05373594199955
Words seen: 9/95; Time taken: 83.40755576900119
Words seen: 10/95; Time taken: 77.31226481000158
Words seen: 11/95; Time taken: 85.55513823600086
Words seen: 12/95; Time taken: 89.35679126300056
Words seen: 13/95; Time taken: 130.77185328700034
Words seen: 14/95; Time taken: 117.28541655799927
Words seen: 15/95; Time taken: 93.31047031299931
Words seen: 16/95; Time taken: 102.00001781300125
Words seen: 17/95; Time taken: 108.2346325210001
Words seen: 18/95; Time taken: 97.67615771700002
Words seen: 19/95; Time taken: 83.201917331
Words seen: 20/95; Time taken: 93.41768364200107
Words seen: 21/95; Time taken: 93.98788159200012
Words seen: 22/95; Time taken: 92.58741949199975
Words seen: 23/95; Time take

E0801 15:08:48.154980 4661413312 session.py:1622] Session failed to close after 30 seconds. Continuing after this point may leave your program in an undefined state.


KeyboardInterrupt: 

In [55]:
word_graph.edges.data()

EdgeDataView([('engineer', 'traffic', {'weight': 1.0, 'edge_emb_wt': 0.40124842524528503}), ('traffic', 'involved', {'weight': 1.0, 'edge_emb_wt': 0.49344101548194885}), ('involved', 'house', {'weight': 1.0, 'edge_emb_wt': 0.3551652133464813}), ('house', 'cracker', {'weight': 1.0, 'edge_emb_wt': 0.36292627453804016}), ('cracker', 'major', {'weight': 1.0, 'edge_emb_wt': 0.3910221457481384}), ('major', 'invoice', {'weight': 1.0, 'edge_emb_wt': 0.296808660030365}), ('invoice', 'life', {'weight': 1.0, 'edge_emb_wt': 0.20629160106182098}), ('life', 'time', {'weight': 1.0, 'edge_emb_wt': 0.7616047263145447}), ('time', 'remember', {'weight': 1.0, 'edge_emb_wt': 0.6120784878730774}), ('time', 'piece', {'weight': 1.0, 'edge_emb_wt': 0.6457200050354004}), ('time', 'day', {'weight': 1.0, 'edge_emb_wt': 0.7213506102561951}), ('remember', 'moment', {'weight': 1.0, 'edge_emb_wt': 0.6937558054924011}), ('moment', 'careful', {'weight': 1.0, 'edge_emb_wt': 0.6300485730171204}), ('careful', 'computer', 

## Testing PageRank

In [56]:
biased_pagerank = nx.pagerank(word_graph, weight="edge_emb_wt")
unbiased_pagerank = nx.pagerank(word_graph)

In [57]:
sorted_biased_rank = sort_by_value(biased_pagerank.items())
sorted_unbiased_rank = sort_by_value(unbiased_pagerank.items())

In [58]:
for i, tup in enumerate(sorted_biased_rank):
    word = tup[0]
    biased_rank = i
    biased_pagerank_score = tup[1]
    word_graph.add_node(word, wt_val=biased_pagerank_score, wt_rank=biased_rank)

In [59]:
for i, tup in enumerate(sorted_unbiased_rank):
    word = tup[0]
    unbiased_rank = i
    unbiased_pagerank_score = tup[1]
    word_graph.add_node(word, val=unbiased_pagerank_score, rank=unbiased_rank)

In [67]:
def get_segment_wise_ranking(word_graph, segment_list):
    for i, sub_text in enumerate(segment_list):
        original_tokens, pos_tuple, filtered_pos_tuple = tp.preprocess_text(sub_text, filter_by_pos=True, stop_words=False)
        sub_keyphrases = gr.get_keyphrases(graph_obj=word_graph, input_pos_text=pos_tuple, post_process=True)
        wt_sub_keyphrase = gr.get_keyphrases(word_graph, input_pos_text=pos_tuple, post_process=True, weight="edge_emb_wt")

        keyphrase_rank_list = []
        for i, phrase_tup in enumerate(wt_sub_keyphrase):
            wt_rank = i
            wt_word = phrase_tup[0]
            for j, un_phrase_tup in enumerate(sub_keyphrases):
                word = un_phrase_tup[0]
                if word == wt_word:
                    orig_rank = j
                    tup = (word, orig_rank, wt_rank, sub_text)
                    keyphrase_rank_list.append(tup)
        
        # print(sub_text)
        yield keyphrase_rank_list

In [75]:
kr = get_segment_wise_ranking(word_graph, segment_list[:2])

In [76]:
print("Word \t\t ===> Index | Original rank | Weighted rank | Difference")
for seg in list(kr):
    print("========New Segment=========")
    print(seg[0][-1])
    for i, (word, orig_rank, wt_rank, text) in enumerate(seg):
        diff = orig_rank - wt_rank
        print("{}".format(word))
        print("{} | {} | {} | {}".format(i, orig_rank, wt_rank, diff))
        print()

Word 		 ===> Index | Original rank | Weighted rank | Difference
========New Segment=========
I was born in involved like many other engineers. I grew reading section how traffic was actually very involved with this house and crackers and little that I know that this works a major invoice on my life did you time section to do you remember the moment when the while basically saving the world asks careful for something computer make itself and the machine just does it and is sales how why the real want know like this example for the okay Google and ask the application service mostly requirement and get the job done just like that today I want to talk about technology based on other which one is a small step in that direction when happens we have an opportunity to like your users and bring more engagement to Europe. However, we can't do it need help as Google. We have been working on important organizing the also information to make a university. The access to help with that we build the k

In [ ]:
for i in word_graph.edges.data():
    print(i[0], i[1], i[2])